In [2]:
import json
import pandas as pd
import duckdb 
import csv

conn = duckdb.connect()
conn

# Step 1: connect to the first duck db file.

In [3]:
database_url = "https://cs.wellesley.edu/~eni/duckdb/2023_wiki_views.duckdb"

# Attach the remote file as a database named 'web_db' and start using it
try:
    conn.execute(f"ATTACH '{database_url}' AS web_db (READ_ONLY);")
    conn.execute("USE web_db;")
    print(f"Successfully attached database from: {database_url}")
except Exception as e:
    print(f"Error attaching database: {e}")

Successfully attached database from: https://cs.wellesley.edu/~eni/duckdb/2023_wiki_views.duckdb


Find out the name of the table.

In [4]:

conn.sql("USE web_db")

conn.sql("SHOW TABLES").show()

┌────────────┐
│    name    │
│  varchar   │
├────────────┤
│ data_table │
└────────────┘



Look at columns.

In [5]:
table_name = "data_table"
query = f"PRAGMA table_info('web_db.{table_name}');"

# We can apply the method .df() to the result of the query to convert it into a dataframe
column_info_df = conn.sql(query).df()
column_info_df

,cid,name,type,notnull,dflt_value,pk
0,0,date,DATE,False,None,False
1,1,country,VARCHAR,False,None,False
2,2,country_code,VARCHAR,False,None,False
3,3,project,VARCHAR,False,None,False
4,4,page_id,BIGINT,False,None,False
5,5,article,VARCHAR,False,None,False
6,6,qid,VARCHAR,False,None,False
7,7,pageviews,BIGINT,False,None,False


Look at the first couple of rows.

In [6]:
query1 = """
SELECT * FROM data_table
LIMIT 10;
"""
result1 = conn.sql(query1).df() # after executing, convert to df for better printout

result1

,date,country,country_code,project,page_id,article,qid,pageviews
0,2023-01-01,United States of America,US,en.wikipedia,100156,Ravi_Shankar,Q103774,966
1,2023-01-01,United States of America,US,en.wikipedia,10239178,John_Galt,Q2492223,451
2,2023-01-01,United States of America,US,en.wikipedia,1024181,Carlos_Tevez,Q50600,529
3,2023-01-01,United States of America,US,en.wikipedia,1025220,Ruth_Buzzi,Q442250,838
4,2023-01-01,United States of America,US,en.wikipedia,10318642,Robert_Smith_(running_back),Q3938426,2347
5,2023-01-01,United States of America,US,en.wikipedia,1039323,Wolfgang_Puck,Q78893,2142
6,2023-01-01,United States of America,US,en.wikipedia,105389,Jeff_Beck,Q192474,1096
7,2023-01-01,United States of America,US,en.wikipedia,10772,Fair_use,Q131562,695
8,2023-01-01,United States of America,US,en.wikipedia,1081037,Gadsden_flag,Q917652,1708
9,2023-01-01,United States of America,US,en.wikipedia,1157583,Anil_Kapoor,Q313956,628


# Step 2: Filter data using qidlist.

In [16]:
with open ('all_articles_by_importance.json') as f:
    health_articles = json.load(f)

In [20]:
truth_df =pd.DataFrame(health_articles)

truth_df = truth_df.dropna(subset=['qid'])

truth_df.head()

,pageid,title,qid,category
0,60380397,996 working hour system,Q62568684,High-importance
1,22570610,Active mobility,Q4010939,High-importance
2,2012158,Aerobic conditioning,Q4688194,High-importance
3,16413778,Ageing,Q332154,High-importance
4,10934212,Air pollution,Q131123,High-importance


In [19]:
qidlist = truth_df['qid'].to_list()
len(qidlist)

3177

How many QIDs are missing?

Transform qidslist into tuples.

In [22]:
qids_tuple = tuple(qidlist)

Talk to the database.

In [23]:
query2 = f"""
SELECT date, page_id, article, qid, pageviews
FROM data_table
WHERE qid IN {qids_tuple}
"""
df_2023 = conn.sql(query2).df()

In [24]:
df_2023.shape

(31708, 5)

Now do the same for 2024.

In [25]:
database_url = 'https://cs.wellesley.edu/~eni/duckdb/2024_wiki_views.duckdb'

# Attach the remote file as a database named 'web_db' and start using it
try:
    conn.execute(f"ATTACH '{database_url}' AS web_db2024 (READ_ONLY);")
    conn.execute("USE web_db2024;")
    print(f"Successfully attached database from: {database_url}")
except Exception as e:
    print(f"Error attaching database: {e}")

Successfully attached database from: https://cs.wellesley.edu/~eni/duckdb/2024_wiki_views.duckdb


In [26]:
query3 = f"""
SELECT date, page_id, article, qid, pageviews
FROM data_table
WHERE qid IN {qids_tuple}
"""
df_2024 = conn.sql(query3).df()

In [27]:
df_2024.shape

(75450, 5)

# Step 3: Merge the 2 dataframes.

In [28]:
df_total = pd.concat([df_2023, df_2024])
df_total.shape

(107158, 5)

In [29]:
df_total.to_csv('all_health_articles.csv')

# Step 4: Make Wikipedia API calls to add more information about each article to each row.

I need to break these up into chunks because otherwise I runinto API errors.

In [30]:
import get_wikidata as f

In [34]:
f.process_qids_to_jsonl(qidlist[0:700], output_filename = 'all_wikidata_1.jsonl')

Starting processing for 700 QIDs.
Results will be written to 'all_wikidata_1.jsonl'.
Processing Q62568684...
Processing Q4010939...
Processing Q4688194...
Processing Q332154...
Processing Q131123...
Processing Q483866...
Processing Q583725...
Processing Q5119833...
Processing Q41583...
Processing Q18206313...
Processing Q5215228...
Processing Q143978...
Processing Q12136...
Processing Q108471361...
Processing Q362854...
Processing Q472094...
Processing Q769815...
Processing Q1215296...
Processing Q19863865...
Processing Q1538635...
Processing Q10302339...
Processing Q60521365...
Processing Q1519838...
Processing Q13156121...
Processing Q870895...
Processing Q16235120...
Processing Q334911...
Processing Q5691094...
Processing Q5691099...
Processing Q530962...
Processing Q1519843...
Processing Q5691262...
Processing Q2617353...
Processing Q21901381...
Processing Q60521539...
Processing Q2764529...
Processing Q5690895...
Processing Q56276811...
Processing Q910866...
Processing Q291019...


In [35]:
f.process_qids_to_jsonl(qidlist[700:1400], output_filename = 'all_wikidata_2.jsonl')

Starting processing for 700 QIDs.
Results will be written to 'all_wikidata_2.jsonl'.
Processing Q27295602...
Processing Q29642721...
Processing Q5523465...
Processing Q131142540...
Processing Q106112358...
Processing Q113021352...
Processing Q2648842...
Processing Q5532583...
Processing Q2575667...
Processing Q124613536...
Processing Q30282508...
Processing Q5549510...
Processing Q5554277...
Processing Q130457575...
Processing Q5234106...
Processing Q86929060...
Processing Q4962188...
Processing Q1529519...
Processing Q5567491...
Processing Q30325118...
Processing Q28148563...
Processing Q22079671...
Processing Q127962835...
Processing Q124741931...
Processing Q5572319...
Processing Q2703769...
Processing Q16840298...
Processing Q5575010...
Processing Q5576809...
Processing Q72797...
Processing Q30314188...
Processing Q5143667...
Processing Q1536234...
Processing Q20736527...
Processing Q5606534...
Processing Q105805550...
Processing Q18150347...
Processing Q54489236...
Processing Q136

In [36]:
f.process_qids_to_jsonl(qidlist[1400:2100], output_filename = 'all_wikidata_3.jsonl')

Starting processing for 700 QIDs.
Results will be written to 'all_wikidata_3.jsonl'.
Processing Q2179394...
Processing Q26709091...
Processing Q2104662...
Processing Q511601...
Processing Q16967850...
Processing Q2529411...
Processing Q17084176...
Processing Q7074848...
Processing Q7074847...
Processing Q7074853...
Processing Q7074854...
Processing Q65042254...
Processing Q7074857...
Processing Q55192753...
Processing Q7074859...
Processing Q7074860...
Processing Q7074861...
Processing Q7074867...
Processing Q7074865...
Processing Q104856816...
Processing Q3143849...
Processing Q629029...
Processing Q380141...
Processing Q122171586...
Processing Q7078029...
Processing Q7079268...
Processing Q104843078...
Processing Q17072404...
Processing Q123046089...
Processing Q7085564...
Processing Q7092706...
Processing Q7093599...
Processing Q22907464...
Processing Q59603...
Processing Q3081320...
Processing Q106412134...
Processing Q17072861...
Processing Q7096383...
Processing Q136445924...
Pro

In [37]:
f.process_qids_to_jsonl(qidlist[2100:2800], output_filename = 'all_wikidata_4.jsonl')

Starting processing for 700 QIDs.
Results will be written to 'all_wikidata_4.jsonl'.
Processing Q16000520...
Processing Q4914652...
Processing Q25203169...
Processing Q4927059...
Processing Q3299670...
Processing Q1072278...
Processing Q450743...
Processing Q16735187...
Processing Q48797772...
Processing Q4969212...
Processing Q4836809...
Processing Q4998417...
Processing Q3045295...
Processing Q185325...
Processing Q5038720...
Processing Q3054053...
Processing Q39087404...
Processing Q797080...
Processing Q5087984...
Processing Q3241451...
Processing Q23074176...
Processing Q96375015...
Processing Q1578...
Processing Q16958381...
Processing Q838858...
Processing Q1099387...
Processing Q863522...
Processing Q5156511...
Processing Q231043...
Processing Q5172791...
Processing Q125352360...
Processing Q89279679...
Processing Q17522146...
Processing Q811154...
Processing Q5201399...
Processing Q2723075...
Processing Q1866767...
Processing Q117484...
Processing Q5204325...
Processing Q52495

In [38]:
f.process_qids_to_jsonl(qidlist[2800:], output_filename = 'all_wikidata_5.jsonl')

Starting processing for 377 QIDs.
Results will be written to 'all_wikidata_5.jsonl'.
Processing Q814819...
Processing Q3398513...
Processing Q27477117...
Processing Q89531596...
Processing Q6550650...
Processing Q128693423...
Processing Q96389388...
Processing Q6619425...
Processing Q134895792...
Processing Q6659294...
Processing Q6659296...
Processing Q33124729...
Processing Q85781994...
Processing Q131470897...
Processing Q135397416...
Processing Q108607689...
Processing Q134238428...
Processing Q2302075...
Processing Q1455470...
Processing Q96218668...
Processing Q6723722...
Processing Q28839711...
Processing Q6736802...
Processing Q109892229...
Processing Q123418243...
Processing Q133843893...
Processing Q119143255...
Processing Q25091279...
Processing Q6744008...
Processing Q30290660...
Processing Q131782932...
Processing Q17143519...
Processing Q26709623...
Processing Q48817527...
Processing Q85875291...
Processing Q179515...
Processing Q6784507...
Processing Q25053642...
Process

# Step 5: make a df with unique qids (NOT raw form).

In [39]:
df_classifier = df_total.groupby(['article', 'qid'], as_index=False)['pageviews'].sum()
df_classifier = df_classifier.rename(columns={'pageviews': 'total_pageviews'})
df_classifier

,article,qid,total_pageviews
0,1977_NestlÃ©_boycott,Q1498774,27259
1,2007_tuberculosis_scare,Q4609852,550
2,2011_United_States_listeriosis_outbreak,Q4622783,871
3,2014_California_Proposition_46,Q119129906,153
4,24_Hour_Fitness,Q4631849,1122
...,...,...,...
878,Yoga_as_exercise,Q65091595,4324
879,Young_Living,Q18067581,2947
880,Zero_emission_zone,Q647266,127
881,Zumba,Q229680,42089


In [41]:
# Load all_wikidata JSONL files into a list of dicts
wikidata = []
with open('all_wikidata_1.jsonl') as f1, open('all_wikidata_2.jsonl') as f2, open('all_wikidata_3.jsonl') as f3, open('all_wikidata_4.jsonl') as f4, open('all_wikidata_5.jsonl') as f5:
    for f in [f1, f2, f3, f4, f5]:
        for line in f:
            wikidata.append(json.loads(line))

# Create a mapping from QID to description
qid_to_description = {item['QID']: item.get('description', None) for item in wikidata}

# Add the description column to df_classifier
df_classifier['description'] = df_classifier['qid'].map(qid_to_description)


In [42]:
df_classifier.head()

,article,qid,total_pageviews,description
0,1977_NestlÃ©_boycott,Q1498774,27259,boycott
1,2007_tuberculosis_scare,Q4609852,550,international health scare
2,2011_United_States_listeriosis_outbreak,Q4622783,871,disease outbreak in the United States
3,2014_California_Proposition_46,Q119129906,153,No description found
4,24_Hour_Fitness,Q4631849,1122,US fitness center chain


Add the category column so the dataset is labeled.

In [43]:
# Load all_articles_by_importance.json
with open('all_articles_by_importance.json') as f:
    articles = json.load(f)

# Create a mapping from QID to category
qid_to_category = {item['qid']: item.get('category', None) for item in articles}

# Add the category column to df_classifier
df_classifier['category'] = df_classifier['qid'].map(qid_to_category)

In [44]:
df_classifier.head()

,article,qid,total_pageviews,description,category
0,1977_NestlÃ©_boycott,Q1498774,27259,boycott,Low-importance
1,2007_tuberculosis_scare,Q4609852,550,international health scare,Low-importance
2,2011_United_States_listeriosis_outbreak,Q4622783,871,disease outbreak in the United States,Low-importance
3,2014_California_Proposition_46,Q119129906,153,No description found,Low-importance
4,24_Hour_Fitness,Q4631849,1122,US fitness center chain,Low-importance


In [48]:
df_classifier['description'] = df_classifier['description'].replace("No description found", None)
df_classifier['category'] = df_classifier['category'].replace("Top-importance", 'High-importance')

In [49]:
df_classifier.tail()

,article,qid,total_pageviews,description,category
878,Yoga_as_exercise,Q65091595,4324,physical activity consisting mainly of yoga poses,High-importance
879,Young_Living,Q18067581,2947,American multi-level marketing company,Low-importance
880,Zero_emission_zone,Q647266,127,defined area,High-importance
881,Zumba,Q229680,42089,dance fitness program,Mid-importance
882,Zyzz,Q2618609,181047,Russian-born Australian bodybuilder and Intern...,Mid-importance


In [54]:
df_classifier.groupby('category')['article'].nunique()

category
High-importance       115
Low-importance        472
Mid-importance        163
Unknown-importance    133
Name: article, dtype: int64

# Step 6: save dataframe as a CSV to be used for text-classification.

In [55]:
df_classifier.to_csv('unique_health_articles.csv')

In [3]:
# Load the CSV files
all_articles = pd.read_csv('all_health_articles.csv')
unique_articles = pd.read_csv('unique_health_articles.csv')

# Ensure the qid columns are named the same
# (rename if necessary, e.g., unique_articles.rename(columns={'QID': 'qid'}, inplace=True))

# Merge to add the category column
merged = all_articles.merge(unique_articles[['qid', 'category']], on='qid', how='left')

# Save the result
merged.to_csv('all_health_articles.csv', index=False)

MergeError: Passing 'suffixes' which cause duplicate columns {'category_x'} is not allowed.

In [4]:
unique_articles.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'article', 'qid', 'total_pageviews',
       'description', 'category', 'page_size', 'revisions_since_2024'],
      dtype='object')

In [5]:
unique_articles.sort_values(by='total_pageviews', ascending=False)

,Unnamed: 0.1,Unnamed: 0,article,qid,total_pageviews,description,category,page_size,revisions_since_2024
676,676,676,Richard_Simmons,Q498019,2949687,American fitness instructor and video producer...,Unknown-importance,82448.0,1202
241,241,241,Enteritis,Q854353,1523654,intestinal inflammatory disease,Mid-importance,19257.0,39
60,60,60,Average_human_height_by_country,Q16128414,1345672,Wikimedia list article,Low-importance,167046.0,523
684,684,684,Sackler_family,Q61911512,1224161,American billionaire family,Low-importance,53256.0,100
478,478,478,Maslow's_hierarchy_of_needs,Q179515,1219581,theory in developmental psychology proposed by...,Unknown-importance,46766.0,140
...,...,...,...,...,...,...,...,...,...
378,378,378,Home_care,Q1642542,91,health care or supportive care provided by a p...,Unknown-importance,6151.0,34
609,609,609,Physical_strength,Q1785966,91,measure of a life form's exertion of force on ...,Low-importance,4971.0,11
624,624,624,Primary_health_care,Q623543,90,essential health care accessible to all indivi...,High-importance,22026.0,12
190,190,190,Decriminalization_of_sex_work,Q17001582,90,removal of criminal penalties for sex work,Low-importance,91810.0,46


In [6]:
all_articles.columns

Index(['Unnamed: 0', 'date', 'page_id', 'article', 'qid', 'pageviews',
       'category'],
      dtype='object')

In [6]:
all_articles.sort_values(by='pageviews', ascending=False)

,Unnamed: 0,date,page_id,article,qid,pageviews,category_x,category_y,category
210517,14736,2024-07-14,147063,Richard_Simmons,Q498019,495422,Unknown-importance,Unknown-importance,Unknown-importance
424212,71161,2024-07-13,147063,Richard_Simmons,Q498019,430067,Unknown-importance,Unknown-importance,Unknown-importance
174170,5319,2024-07-15,147063,Richard_Simmons,Q498019,183777,Unknown-importance,Unknown-importance,Unknown-importance
188286,8958,2024-12-04,1845551,UnitedHealth,Q2103926,167186,High-importance,High-importance,High-importance
188295,8958,2024-12-04,1845551,UnitedHealth,Q2103926,167186,High-importance,High-importance,High-importance
...,...,...,...,...,...,...,...,...,...
228374,19327,2024-01-15,27000,Smog,Q169994,90,Mid-importance,Mid-importance,Mid-importance
228373,19327,2024-01-15,27000,Smog,Q169994,90,Mid-importance,Mid-importance,Mid-importance
228372,19327,2024-01-15,27000,Smog,Q169994,90,Mid-importance,Mid-importance,Mid-importance
228371,19327,2024-01-15,27000,Smog,Q169994,90,Mid-importance,Mid-importance,Mid-importance


# ADDITIONAL: Get page size, number of revisions since 2024.

In [7]:
import mwclient
from tqdm import tqdm

site = mwclient.Site('en.wikipedia.org')

In [ ]:
def extractExtraProps(article_title):
    try:
        page = site.Pages[article_title]
        # Page size in bytes
        page_size = page.length

        # Count revisions since 2024-01-01
        rev_count = 0
        for rev in page.revisions(start='2024-01-01T00:00:00Z', dir='newer'):
            rev_count += 1

        return page_size, rev_count
    except Exception as e:
        print(f"Error for {article_title}: {e}")
        return None, None

In [ ]:
unique_articles['page_size'] = None
unique_articles['revisions_since_2024'] = None

for idx, row in tqdm(unique_articles.iterrows(), total=len(unique_articles)):
    page_size, rev_count = extractExtraProps(row['article'])
    unique_articles.at[idx, 'page_size'] = page_size
    unique_articles.at[idx, 'revisions_since_2024'] = rev_count

 32%|███▏      | 285/883 [01:34<02:22,  4.21it/s]

Now we can export this and use it to train the classifier.

In [9]:
unique_articles.head()

,Unnamed: 0,article,qid,total_pageviews,description,category,page_size,revisions_since_2024
0,0,1977_NestlÃ©_boycott,Q1498774,27259,boycott,Low-importance,None,0
1,1,2007_tuberculosis_scare,Q4609852,550,international health scare,Low-importance,16565,13
2,2,2011_United_States_listeriosis_outbreak,Q4622783,871,disease outbreak in the United States,Low-importance,23708,13
3,3,2014_California_Proposition_46,Q119129906,153,NaN,Low-importance,5373,5
4,4,24_Hour_Fitness,Q4631849,1122,US fitness center chain,Low-importance,21019,53


In [10]:
unique_articles.to_csv('unique_health_articles.csv')

# EVEN MORE.

In [8]:
def extractMoreProps(article_title):
    page = site.Pages[article_title]
    try:
        # incoming links
        linking_pages = list(page.backlinks(namespace=0))
        incoming_number = len(linking_pages)

        # outgoing links
        internal_links = list(page.links())
        internal_count = len(internal_links)
        external_links = list(page.extlinks())
        external_count = len(external_links)

        outgoing_number = internal_count+external_count 
        return incoming_number, outgoing_number
    except Exception as e:
        print(f"An error occurred: {e}")
        return None, None

In [9]:
unique_articles['incoming_number'] = None
unique_articles['outgoing_number'] = None

for idx, row in tqdm(unique_articles.iterrows(), total=len(unique_articles)):
    in_n, out_n = extractMoreProps(row['article'])
    unique_articles.at[idx, 'incoming_number'] = in_n
    unique_articles.at[idx, 'outgoing_number'] = out_n

100%|██████████| 883/883 [11:15<00:00,  1.31it/s]


In [10]:
unique_articles.head()

,Unnamed: 0.1,Unnamed: 0,article,qid,total_pageviews,description,category,page_size,revisions_since_2024,incoming_number,outgoing_number
0,0,0,1977_NestlÃ©_boycott,Q1498774,27259,boycott,Low-importance,NaN,0,0,0
1,1,1,2007_tuberculosis_scare,Q4609852,550,international health scare,Low-importance,16565.0,13,41,83
2,2,2,2011_United_States_listeriosis_outbreak,Q4622783,871,disease outbreak in the United States,Low-importance,23708.0,13,228,305
3,3,3,2014_California_Proposition_46,Q119129906,153,NaN,Low-importance,5373.0,5,957,1046
4,4,4,24_Hour_Fitness,Q4631849,1122,US fitness center chain,Low-importance,21019.0,53,88,88


In [11]:
unique_articles.to_csv('unique_health_articles.csv')